---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [53]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [54]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [102]:
def get_list_of_university_towns():
    
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    data = []
    state = None
    state_towns = []
    with open('university_towns.txt') as file:
        for line in file:
            thisLine = line[:-1]
            if thisLine[-6:] == '[edit]':
                state = thisLine[:-6]
                continue
            if '(' in line:
                town = thisLine[:thisLine.index('(')-1]
                state_towns.append([state,town])
            else:
                town = thisLine
                state_towns.append([state,town])
            data.append(thisLine)
    df = pd.DataFrame(state_towns,columns = ['State','RegionName'])
    return df

In [103]:
def read_gdp(): 
    #only look at GDP data from the first quarter of 2000 onward.
    #(use the chained value in 2009 dollars)
    #Only needs column 5&7
    gdp = pd.read_excel('gdplev.xls', skiprows = 4)
    
    # print(gdp)
    #gdp = gdp[gdp['Unnamed: 4','Unnamed: 6']]
    gdp = gdp.iloc[:,4:7]
    gdp.drop(['Unnamed: 5'], axis=1, inplace=True)
    gdp = gdp.loc[215:]
    gdp.rename(columns={'Unnamed: 4':'Year & Quarter', 'Unnamed: 6':'GDP'}, inplace=True)
    gdp.reset_index(drop=True, inplace=True)
    
    return gdp

In [104]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp = read_gdp()
    
    #All of the years that has had a declinge in GDP-growth
    gdp['GDP-DIFF'] = gdp['GDP'].diff()
    gdp = gdp.where(gdp['GDP-DIFF'] < 0)
    gdp.dropna(inplace = True)
    
    #Finding the year that had had two following
    #years of decline in growth 
    gdp['index'] = gdp.index
    gdp['index-diff'] = gdp['index'].diff()
    gdp_min = gdp['index-diff'].idxmin()
    
    
    return gdp['Year & Quarter'].loc[gdp_min-1]

In [105]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp = read_gdp()
    gdp['GDP-diff'] = gdp['GDP'].diff()
    gdp.dropna(inplace = True)
    
    gdp = gdp[gdp['GDP-diff']>0]
    gdp['index'] = gdp.index
    gdp['index-diff'] = gdp['index'].diff()
    
    max_idx = gdp['index-diff'].idxmax()
 
    return gdp['Year & Quarter'].loc[max_idx+1]

In [106]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdp = read_gdp()
    max_idx = gdp[gdp['Year & Quarter']==get_recession_end()].index[0]
    min_idx = gdp[gdp['Year & Quarter']==get_recession_start()].index[0]
    gdp = gdp.iloc[min_idx: max_idx]
    
    bott = gdp['GDP'].min()
    test = gdp[gdp['GDP']==bott].index[0]
    
    return gdp['Year & Quarter'].loc[test]

In [107]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows'''
    
    hous = pd.read_csv('City_Zhvi_AllHomes (1).csv')    
    
    hous.drop(hous.loc[:, 'Metro':'1999-12'].columns, axis=1, inplace=True)
    hous.drop(['RegionID'], axis=1, inplace=True)
    #print(states)
    
    #Skapar en ny df där jag kan föra över ny ifo
    new_hous = pd.DataFrame(hous[['State', 'RegionName']])
    
    for year in range(2000,2016): 
        new_hous[str(year)+'q1'] = hous[[str(year)+'-01',str(year)+'-02',str(year)+'-03']].mean(axis=1) 
        new_hous[str(year)+'q2'] = hous[[str(year)+'-04',str(year)+'-05',str(year)+'-06']].mean(axis=1) 
        new_hous[str(year)+'q3'] = hous[[str(year)+'-07',str(year)+'-08',str(year)+'-09']].mean(axis=1) 
        new_hous[str(year)+'q4'] = hous[[str(year)+'-10',str(year)+'-11',str(year)+'-12']].mean(axis=1) 
    
    #In new_hous: missing year 2016-01 : 2016-08. 
    year = 2016
    
    new_hous[str(year)+'q1'] = hous[[str(year)+'-01',str(year)+'-02',str(year)+'-03']].mean(axis=1)
    new_hous[str(year)+'q2'] = hous[[str(year)+'-04',str(year)+'-05',str(year)+'-06']].mean(axis=1)
    new_hous[str(year)+'q3'] = hous[[str(year)+'-07',str(year)+'-08']].mean(axis=1)
    
    new_hous['State'].replace(states, inplace=True)
    new_hous.set_index(['State', 'RegionName'], inplace = True)   
    
    return new_hous


In [108]:

def run_est():
    '''
    1. First creates new data showing the decline or growth of housing prices
        between the recession start and the recession bottom. 
    
    2. Then runs a ttest comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    uni = get_list_of_university_towns()
    gdp = read_gdp()
    hous = convert_housing_data_to_quarters()
    
    start=get_recession_start()
    end=get_recession_end()
    bottom = get_recession_bottom()
    
    #För punkt 1. 
    hous['diff'] = hous[start]-hous[end]
    hous = hous[[start, bottom, 'diff']]
    hous.reset_index(inplace=True)
    
    #punkt 2
    
    #EJ uni-städer
    nonuni = hous.drop(uni.index) #Bara housing-prices som INTE är universitetsstäder! 
    
    #UNI-Städer
    unidata = hous.merge(uni, right_index=True, left_index=True, how='inner')
    
    # T-TEST
    p, t = ttest_ind(nonuni['diff'].dropna(), unidata['diff'].dropna())
    
    if p<0.01:
        different = True 
    else: 
        different = False 
    
    if unidata['diff'].mean() > nonuni['diff'].mean():
        better = 'university town'
    else: 
        better = "non-university town"
    
    return different, p, better
